#Ejemplo 2: El  dataset MINST, reconocimiento de carácteres
---



**Objetivo y comprensión del problema**

El objetivo de este problema es la clasificación de caracteres de números escritos a mano. Este es el primer ejemplo que se utiliza habitualmente y se puede ver su descripción original en el siguiente [enlace](https://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/2.1-a-first-look-at-a-neural-network.ipynb).

Se trata de clasificar imágenes en escala de grises (de 28x28px) correspondientes a los números del 0 al 9.

El conjunto de datos a utilizar se denomina MNIST y consiste en 60000 imágenes de entrenamiento y otras 10000 de test y fue creado por el NIST en los 80. 

Este ejemplo se considera el "Hola Mundo" de Deep learning.

## Paso 1: Gestión de los datos




Para la configuración de las bibliotecas a importar, se incluye Keras y el conjunto de datos que ya viene incluido.

In [1]:
import keras
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical




Using TensorFlow backend.


**1.1-Carga de datos**

En este caso, están disponibles como parte de la biblioteca de Keras.

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


**1.2-Visualización de los datos**

*   Se puede comprobar la forma que tienen nuestros datos. En este caso, 60000 imágenes de 28x28 px.


In [3]:
train_images.shape


(60000, 28, 28)

In [4]:
#Mostrar la forma de las etiquetas.

train_labels


array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

**1.3-Codificar los datos**

En este caso se asegura que los datos son todos numéricos convirtiendo el valor inicial. 

En concreto, se escalan los valores entre [0,1], inicialmente se trataba de un array (60000, 28, 28)  con valores entre 0-255. Ahora, se convierte en un array (60000, 28 * 28) con valores entre 0 y 1.


In [0]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

Adicionalmente, categorizamos las clases de salida (0-9).

Se utiliza la función to_categorical de Keras:

*Converts a class vector (integers) to binary class matrix. E.g. for use with categorical_crossentropy.*

In [0]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
#Mostrar la forma de las etiquetas.

train_labels


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

**1.4-Seleccionar los datos**

En este caso, los datos ya están preparados y divididos apropiadamente al descargar el dataset.





## Paso 2: Arquitectura e implementación de nuestra red


1.   La entrada de nuestra red será una capa con 512 nodos y con una forma de 28x28 (de acuerdo al tamaño de la imagen).
2.   La función de activación o filtrado de esta primera capa será relu. Esta capa es "densa" o "totalmente conectada"
3.   La función de activación en la capa de salida será softmax. Esta capa es "densa" o "totalmente conectada" con 10 posibles valores de salida siendo la interpretación la probabilidad de que sea uno de los números escritos.
4. La función de pérdida será **categorical_crossentropy** para realizar la función de pérdida en problemas de clasificación multi-clase.
5. La función de optimización **rmsprop**. Se trata de una función gradiente, que aunque no ha sido [ publicada oficialmente](http://ruder.io/optimizing-gradient-descent/index.html#rmsprop), se ha utilizado ampliamente y que busca dirigirse hacia el gradiente de la separación más óptima de clases.

Se puede interpretar como:

*Changes to the weights during training are now not purely in the direction of the gradient, but rather in the direction of the elementwise division of the gradient by this vector you are maintaining.* 

6. Métricas: en este caso se selecciona sólo la precisión.




In [9]:
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

## Paso 3:  Entrenamiento



En este caso el entrenamiento se realiza buscando encajar las imágenes de entrenamiento a sus clases.

In [0]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
60000/60000 [==============================] - 6s 94us/step - loss: 0.2580 - acc: 0.9250
Epoch 2/5
60000/60000 [==============================] - 6s 93us/step - loss: 0.1053 - acc: 0.9689
Epoch 3/5
60000/60000 [==============================] - 6s 95us/step - loss: 0.0702 - acc: 0.9791
Epoch 4/5
60000/60000 [==============================] - 6s 96us/step - loss: 0.0505 - acc: 0.9846
Epoch 5/5
60000/60000 [==============================] - 5s 89us/step - loss: 0.0385 - acc: 0.9887


Como resultado se pueder ver que las 5 iteraciones de entrenamiento rápidamente se consiguen buenos resultados:

*  Precisión de 98.87%



## Paso 4:  Test

En este caso, se va evaluar cómo el conjunto de test se predice con el modelo entrenado.

In [0]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 57us/step


Una vez que se ha realizado la evaluación de las imágenes de test, se procede a mostrar la precisión (que se había seleccionado al construir el modelo)

In [0]:
print('Precisión:', test_acc)

Precisión: 0.9792


En este caso el valor de precisión en test es menor que en entrenamiento lo que implica un problema de "overfitting". 

Nuestro modelo se comportará peor con nuevos datos que en entrenamiento.

#Tareas

1.   Investigar cómo evitar el overfitting (ej: dropout).
2.   Cambiar el tipo de la función de pérdida y ver cómo afecta al resultado.
3.   Cambiar la función de optimización y ver cómo afecta al resultado.

#Otras Referencias


1.   https://towardsdatascience.com/preventing-deep-neural-network-from-overfitting-953458db800a


